In [ ]:
hasattr(object, name)   
#判断object中是否有name属性，返回BOOL值，有name特性返回True， 否则返回False,name要用括号括起来

In [1]:
# 设置用户代理
from urllib.request import Request,urlopen
from urllib import error
import re

In [2]:
def download(url, user_agent = 'wswp', num_retries=2):
    print('Downloading:', url)
    headers = {'User-agent': user_agent}
    request = Request(url, headers = headers)
    try:
        html = urlopen(request).read()
    except error.URLError as e:
        print('Download error:',e.reason)
        html = None
        if num_retries > 0:
            if hasattr(e, 'code') and 500 <= e.code < 600:
                # recursively retry 5xx HTTP errors
                return download(url, user_agent,num_retries-1)
    return html
#print(download('http://example.webscraping.com/'))    
print(download('http://httpstat.us/500'))


Downloading: http://httpstat.us/500
Download error: [Errno 11004] getaddrinfo failed
None


## 网站地图爬虫

## 中止爬虫运行用 Ctrl + C , or CMD + C

In [3]:
import itertools
import urllib.request
from urllib.error import URLError, HTTPError, ContentTooShortError


def download(url, user_agent='wswp', num_retries=2, charset='utf-8'):
    print('Downloading:', url)
    request = urllib.request.Request(url)
    request.add_header('User-agent', user_agent)
    try:
        resp = urllib.request.urlopen(request)
        cs = resp.headers.get_content_charset()
        if not cs:
            cs = charset
        html = resp.read().decode(cs)
    except (URLError, HTTPError, ContentTooShortError) as e:
        print('Download error:', e.reason)
        html = None
        if num_retries > 0:
            if hasattr(e, 'code') and 500 <= e.code < 600:
                # recursively retry 5xx HTTP errors
                return download(url, num_retries - 1)
    return html

'''
def crawl_site(url, max_errors=5):
    num_errors = 0
    for page in itertools.count(1):
        pg_url = '{}{}'.format(url, page)
        html = download(pg_url)
        if html is None:
            num_errors += 1
            if num_errors == max_errors:
                # reached max number of errors, so exit
                break
        else:
            num_errors = 0
            # success - can scrape the result
           
   
'''
def crawl_sitemap(url):
    #sitemap = download(url).decode('utf-8') #download the sitemap file
    sitemap = download(url)
    #sitemap = sitemap.decode('utf-8')
    links = re.findall('<loc>(.*?)</loc>', sitemap) # extract the sitemap links
   
    
    for link in links: # download each link
        
        html = download(link) # scarpe html here'''
        
#crawl_sitemap('http://example.webscraping.com/sitemap.xml')

## ID iteration crawler

In [39]:
def crawl_site(url):
    for page in itertools.count(1):  #count() 接收两个参数，第一个参数指定开始值，默认为 0，第二个参数指定步长，默认为 1：
        pg_url = '{}{}'.format(url, page)
        html = download(pg_url)
        if html is None:
            break
        # success - can scrape the result

In [40]:
print(crawl_site('http://example.webscraping.com/-'))

Downloading: http://example.webscraping.com/-1
Downloading: http://example.webscraping.com/-2
Downloading: http://example.webscraping.com/-3
Downloading: http://example.webscraping.com/-4
Downloading: http://example.webscraping.com/-5
Downloading: http://example.webscraping.com/-6
Downloading: http://example.webscraping.com/-7
Downloading: http://example.webscraping.com/-8
Downloading: http://example.webscraping.com/-9
Downloading: http://example.webscraping.com/-10
Downloading: http://example.webscraping.com/-11
Downloading: http://example.webscraping.com/-12
Download error: TOO MANY REQUESTS
None


In [ ]:
## 设置发生错误下载次数

In [35]:
def crawl_site(url, max_errors=5):
    num_errors = 0
    for page in itertools.count(1):
        pg_url = '{}{}'.format(url, page)
        html = download(pg_url)
        if html is None:
            num_errors += 1
            if num_errors == max_errors:
                # reached max number of errors, so exit
                break
        else:
            num_errors = 0
            # success - can scrape the result

In [36]:
print(crawl_site('http://example.webscraping.com/-'))

Downloading: http://example.webscraping.com/-1
Downloading: http://example.webscraping.com/-2
Downloading: http://example.webscraping.com/-3
Downloading: http://example.webscraping.com/-4
Downloading: http://example.webscraping.com/-5
Downloading: http://example.webscraping.com/-6
Downloading: http://example.webscraping.com/-7
Downloading: http://example.webscraping.com/-8
Downloading: http://example.webscraping.com/-9
Downloading: http://example.webscraping.com/-10
Downloading: http://example.webscraping.com/-11
Downloading: http://example.webscraping.com/-12
Download error: TOO MANY REQUESTS
Downloading: http://example.webscraping.com/-13
Download error: TOO MANY REQUESTS
Downloading: http://example.webscraping.com/-14
Download error: TOO MANY REQUESTS
Downloading: http://example.webscraping.com/-15
Download error: TOO MANY REQUESTS
Downloading: http://example.webscraping.com/-16
Download error: TOO MANY REQUESTS
None


In [ ]:
## 链接爬虫

In [4]:


'''
def download(url, user_agent='wswp', num_retries=2, charset='utf-8'):
    print('Downloading:', url)
    request = urllib.request.Request(url)
    request.add_header('User-agent', user_agent)
    try:
        resp = urllib.request.urlopen(request)
        cs = resp.headers.get_content_charset()
        if not cs:
            cs = charset
        html = resp.read().decode(cs)
    except (URLError, HTTPError, ContentTooShortError) as e:
        print('Download error:', e.reason)
        html = None
        if num_retries > 0:
            if hasattr(e, 'code') and 500 <= e.code < 600:
                # recursively retry 5xx HTTP errors
                return download(url, num_retries - 1)
    return html
'''

def link_crawler(start_url, link_regex):
    " Crawl from the given start URL following links matched by link_regex "
    crawl_queue = [start_url]
    # keep track which URL's have seen before
    seen = set(crawl_queue)
    while crawl_queue:
        url = crawl_queue.pop()
        html = download(url)
        if not html:
            continue
        # filter for links matching our regular expression
        for link in get_links(html):
            if re.match(link_regex, link):
                abs_link = urljoin(start_url, link)
                if abs_link not in seen:
                    seen.add(abs_link)
                    crawl_queue.append(abs_link)


def get_links(html):
    # Return a list of links from html "
    # a regular expression to extract all links from the webpage
    webpage_regex = re.compile("""<a[^>]+href=["'](.*?)["']""", re.IGNORECASE)
    # list of all links from the webpage
    return webpage_regex.findall(html)


In [7]:
link_crawler('http://example.webscraping.com','/(index|view)/')

Downloading: http://example.webscraping.com


In [1]:
g= (x*x for  x in range(10))
g

<generator object <genexpr> at 0x000000000524C308>

In [4]:
for n in g:
    print(n)

1
4
9
16
25
36
49
64
81
